## Charts

In [2]:
import pandas as pd
import numpy as np

In [3]:
nat_hir = pd.read_csv('natl_home_inc_hir_8918.csv')  # Zillow (summary) + Census annual inc
ca_hir = pd.read_csv('ca_state_home_inc_hir_8918.csv')  # Zillow (summary) + Census annual inc
reg_hir = pd.read_csv('reg_home_inc_hir_8918.csv')  # FHFA home + SAIPE inc (reg_hirs_8918.py)
county_hir = pd.read_csv('county_home_inc_hir_8918.csv')  # FHFA home + SAIPE inc
county_units = pd.read_csv('county_housing_units_8918.csv')  # Census unit counts
hh_count_hir = pd.read_csv('hh_count_hir_8918.csv')
hh_count_per_inc5 = pd.read_csv('hh_count_housing_per_inc_5yr_0816.csv')
hh_count_per_inc1 = pd.read_csv('hh_count_housing_per_inc_1yr_1018.csv')
fed = pd.read_csv('fed_int_rate_9020.csv')

nat1 = pd.read_csv('national_rent_vs_mort_and_inc_1yr.csv')
nat5 = pd.read_csv('national_rent_vs_mort_and_inc_5yr.csv')
region1 = pd.read_csv('region_rent_vs_mort_and_inc_1yr.csv')
region5 = pd.read_csv('region_rent_vs_mort_and_inc_5yr.csv')
county1 = pd.read_csv('county_rent_mort_inc_units_1yr.csv')
county5 = pd.read_csv('county_rent_mort_inc_units_5yr.csv')
rent_share_inc_pop_den5 = pd.read_csv('rent_share_inc_by_pop_den_5yr_0816.csv')
rent_share_inc_pop_den1 = pd.read_csv('rent_share_inc_by_pop_den_1yr_1018.csv')
mort_share_inc_pop_den5 = pd.read_csv('mort_share_inc_by_pop_den_5yr_0816.csv')
mort_share_inc_pop_den1 = pd.read_csv('mort_share_inc_by_pop_den_1yr_1018.csv')
rent_share_inc_hous_den5 = pd.read_csv('rent_share_inc_by_hous_den_5yr_0816.csv')
rent_share_inc_hous_den1 = pd.read_csv('rent_share_inc_by_hous_den_1yr_1018.csv')
mort_share_inc_hous_den5 = pd.read_csv('mort_share_inc_by_hous_den_5yr_0816.csv')
mort_share_inc_hous_den1 = pd.read_csv('mort_share_inc_by_hous_den_1yr_1018.csv')
sold_new_homes = pd.read_excel('sold_cust.xls', skiprows=8, skipfooter=4)
reg_inc = pd.read_csv('reg_incs_census_summary.csv').sort_values(['Year', 'Region'])
reg_home = pd.read_csv('irs_zillow_reg_med_home_9618.csv').sort_values(['Year', 'Region'])

In [4]:
%run chart_theme.py

---

### National Med. Housing % of Income

In [158]:
# Supplementing 1-yr data w/ 08 & 09 from 5-yr
nat_comb = pd.concat([nat5[nat5.Year < 2010], nat1])

In [159]:
nat_mort_text_df = pd.DataFrame({'Year':(range(2008, 2011)), 
                                 'Numbers':[0,0,40], 
                                 'Text':['Renters', 'Homeowners', '_'*76]})

In [161]:
nat_mort = alt.Chart(nat_comb
                    ).mark_line(size=7, strokeCap='round', color='#97c2f2', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008,2018], ticks=True, tickSize=13, 
                                                                        labelFontSize=23, labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Mort_Share_Inc_whole',
                                               axis=alt.Axis(values=list(range(15,40,5)), 
                                                             labelPadding=30, 
                                                             labelOpacity=.7,
                                                             labelFontSize=23), 
                                               scale=alt.Scale(domain=[10,40])))

In [176]:
nat_rent = alt.Chart(nat_comb, title='Income Devoted to Housing'
                    ).mark_line(size=7, strokeCap='round', color='#cede54', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008,2018], ticks=True, tickSize=13, labelFontSize=23, 
                                                                        labelOpacity=.7, labelPadding=13)), 
                                        y=alt.Y('Rent_Share_Inc_whole', title='Housing expenses have trended slightly downwards since 2010',
                                               axis=alt.Axis(values=list(range(15,40,5)), 
                                                             titleX=-31, titleY=-40, titleOpacity=.7, titleFontSize=26,
                                                             labelPadding=30, labelOpacity=.7, labelFontSize=23),
                                               scale=alt.Scale(domain=[10,40])))

renters = alt.Chart(nat_mort_text_df.query('Year == 2008')).mark_text(dx=615, dy=-418, font='lato', fontSize=32, color='#b7c83e').encode(x='Year:O', y='Numbers', text='Text')
homeowners = alt.Chart(nat_mort_text_df.query('Year == 2009')).mark_text(dx=600, dy=-225, font='lato', fontSize=32, color='#81addc').encode(x='Year:O', y='Numbers', text='Text')
line = alt.Chart(nat_mort_text_df.query('Year == 2010')).mark_text(dx=223, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='Year:O', y='Numbers', text='Text')
y_labels_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=17, dy=0, font='lato', fontSize=23, opacity=.7).encode(x='x:O', y='y', text='text')

In [177]:
nat_comb_plot = (nat_mort + nat_rent + renters + homeowners + y_labels + line).configure_title(fontSize=32)
nat_comb_plot

alt.LayerChart(...)

In [178]:
nat_comb_plot.save('nat_rent_mort_per_inc.png', scale_factor=3)

---

### National Median HIR

In [196]:
natl_med = alt.Chart(nat_hir, title='Home Value Relative to Annual Income'
                    ).mark_line(size=6, strokeCap='round', color='#4c92d8',interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(ticks=True, tickSize=10, values=[1990, 2000, 2010, 2018], 
                                                                        labelOpacity=.7, labelFontSize=18)), 
                                        y=alt.Y('HIR', title='National median home value / median annual household income', 
                                                scale=alt.Scale(domain=[2,4.5]), 
                                                axis=alt.Axis(values=list(np.arange(2.5,4.5,.5)), 
                                                              titleX=-30, titleY=-15, titleOpacity=.7, titleFontSize=20, 
                                                              labelOpacity=.7, labelFontSize=18, labelPadding=30)))

y_text_df = pd.DataFrame({'x':[1989], 'y':[4.5], 'text':['4.5 times income']})
y_text = alt.Chart(y_text_df).mark_text(dx=28, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2004], 'y':[4.5], 'text':['_'*77]})).mark_text(dx=45, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='x:O', y='y', text='text')


In [197]:
natl_med_plot = (natl_med + y_text + line)
natl_med_plot

alt.LayerChart(...)

In [198]:
natl_med_plot.save('natl_med_hir.png', scale_factor=3)

---

### Regional Med. HIR

Homeowners:

In [4]:
reg_comb = pd.concat([region5[region5.Year < 2010], region1])
reg_mort_text_df = pd.DataFrame({'Year':(range(2008, 2019)), 
                                 'Numbers':[round(i,2) for i in np.arange(0,.55,.05)], 
                                 'Text':(['West', 'Midwest', 'South', 'Northeast'] + list(np.full(7, np.nan)))})

In [73]:
reg_mort = alt.Chart(reg_comb, title='Homeowner Income Devoted to Housing'
                    ).mark_line(size=7, strokeCap='round', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, labelFontSize=24, labelOpacity=.7)), 
                                       y=alt.Y('Mort_Share_Inc_whole', 
                                               title='Regional comparison', 
                                               axis=alt.Axis(values=list(range(15,40,5)), 
                                                             titleX=-30, 
                                                             titleOpacity=.7,
                                                             titleY=-40,
                                                             titleFontSize=26,
                                                             labelFontSize=24,
                                                             labelPadding=30, 
                                                             labelOpacity=.7), 
                                               scale=alt.Scale(domain=[10,40])),  
                                       color=alt.Color('Region', title='',
                                                       scale=alt.Scale(domain=['Northeast', 'West', 'South', 'Midwest'],
                                                                       range=['#00a58a', '#004d66', '#ffd5b0', '#c86300'])))

line = alt.Chart(pd.DataFrame({'x':[2013], 'y':[40], 'text':['_'*76]})).mark_text(dx=59, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='x:O', y='y', text='text')
y_labels_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=21, dy=0, font='lato', fontSize=24, opacity=.7).encode(x='x:O', y='y', text='text')

reg_mort_plot = reg_mort + y_labels + line
reg_mort_plot = reg_mort_plot.configure_title(fontSize=30).configure_legend(orient='none', legendX=605, legendY=55, labelFont='lato', labelFontSize=30, labelOpacity=.7, labelFontWeight='normal', symbolSize=200, symbolStrokeWidth=7)

In [74]:
reg_mort_plot

alt.LayerChart(...)

In [75]:
reg_mort_plot.save('reg_homeower_inc_per_housing.png', scale_factor=3)

Renters:

In [70]:
reg_rent = alt.Chart(reg_comb, title='Renter Income Devoted to Housing'
                    ).mark_line(size=7, strokeCap='round', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, labelFontSize=24, labelOpacity=.7)), 
                                       y=alt.Y('Rent_Share_Inc_whole', 
                                               title='Regional comparison', 
                                               axis=alt.Axis(values=list(range(15,40,5)), 
                                                             titleX=-30, 
                                                             titleOpacity=.7,
                                                             titleFontSize=26,
                                                             titleY=-40,
                                                             labelFontSize=24,
                                                             labelPadding=30, 
                                                             labelOpacity=.7), 
                                               scale=alt.Scale(domain=[10,40])), 
                                       color=alt.Color('Region', title='',
                                                       scale=alt.Scale(domain=['Northeast', 'West', 'South', 'Midwest'], 
                                                                       range=['#00a58a', '#004d66', '#f9d8a8', '#c86300'])))
line = alt.Chart(pd.DataFrame({'x':[2013], 'y':[40], 'text':['_'*76]})).mark_text(dx=59, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x='x:O', y='y', text='text')
y_labels_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=21, dy=0, font='lato', fontSize=24, opacity=.7).encode(x='x:O', y='y', text='text')

reg_rent_plot = reg_rent + y_labels + line
reg_rent_plot = reg_rent_plot.configure_title(fontSize=30).configure_legend(orient='none', legendX=605, legendY=55, labelFont='lato', labelFontSize=30, labelOpacity=.7, labelFontWeight='normal', symbolSize=200, symbolStrokeWidth=7)

In [72]:
reg_rent_plot

alt.LayerChart(...)

In [76]:
reg_rent_plot.save('reg_renter_inc_per_housing.png', scale_factor=3)

---

### HIR by Region:

In [ ]:
reg_inc = reg_inc[reg_inc.Year > 1995]
reg_home.Med_Home = reg_home.Med_Home.astype(int)

In [7]:
inc_home = reg_inc.copy()
inc_home['Med_Home'] = reg_home['Med_Home'].values
inc_home['HIR'] = inc_home.Med_Home / inc_home.Med_Inc

In [188]:
region_text_df = pd.DataFrame({'Year':(range(1996, 2019)), 
                               'Numbers':[6,4.1,3.3,2.6] +list(range(7,26)), 
                               'Text':(['West', 'Northeast', 'South', 'Midwest'] + list(np.full(19, np.nan)))})

In [189]:
chart = alt.Chart(inc_home, title='Home Value Relative to Annual Income').mark_line(size=7, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O', axis=alt.Axis(values=[2000, 2010, 2018], ticks=True, tickSize=10, labelFontSize=24, labelOpacity=.7)), 
    y=alt.Y('HIR', title='Regional comparison', 
            axis=alt.Axis( titleY=-40, titleFontSize=26, titleOpacity=.7, labelFontSize=24, labelOpacity=.7, values=list(range(2,7,2)), format='a'),
            scale=alt.Scale(domain=[0,8])),
    color=alt.Color('Region', title='', scale=alt.Scale(domain=['West', 'Northeast', 'South', 'Midwest'], range=['#004d66', '#00a58a', '#ffd5b0', '#c86300'])))

line = alt.Chart(pd.DataFrame({'x':[2000], 'y':[8], 'text':['_'*74]})).mark_text(dx=245, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')
y_labels_df = pd.DataFrame({'x':[2000], 'y':[8], 'text':['8 times income']})
y_label = alt.Chart(y_labels_df.query('x == 2000')).mark_text(dx=-57, dy=0, font='lato', fontSize=24, opacity=.7).encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')

chart = chart + line + y_label
chart = chart.configure_title(fontSize=30).configure_legend(orient='none', legendX=605, legendY=150, labelFont='lato', labelFontSize=30, labelOpacity=.7,  labelFontWeight='normal', symbolSize=200, symbolStrokeWidth=7)

In [190]:
chart

alt.LayerChart(...)

In [191]:
chart.save('reg_hir_zillow_zips_9618.png', scale_factor=3)

Old (county-based calculations):

In [78]:
"""region_text_df = pd.DataFrame({'Year':(range(1989, 2019)), 
                               'Numbers':(list(range(30))), 
                               'Text':(['West', 'Northeast', 'South', 'Midwest'] + list(np.full(26, np.nan)))})"""

In [79]:
"""reg_chart = alt.Chart(reg_hir, title='Home Value Relative to Annual Income').mark_line(size=7, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O', axis=alt.Axis(values=[1990, 2000, 2010, 2018], ticks=True, tickSize=10, labelFontSize=22, labelOpacity=.7)), 
    y=alt.Y('HIR', title='Regional comparison', axis=alt.Axis(labelFontSize=22, labelOpacity=.7, values=list(range(2,7)), format='a'), scale=alt.Scale(domain=[1,7.5])),
    color=alt.Color('Region', legend=None, scale=alt.Scale(domain=['West', 'Northeast', 'South', 'Midwest'], range=['#004d66', '#00a58a', '#ffd5b0', '#c86300'])))

west = alt.Chart(region_text_df.query('Numbers == 0')
                   ).mark_text(dx=31, dy=-330, fontSize=28, fontWeight='normal', color='#004d66'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
northeast = alt.Chart(region_text_df.query('Numbers == 1')
                   ).mark_text(dx=60, dy=-199, fontSize=28, fontWeight='normal', color='#00a58a'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
south = alt.Chart(region_text_df.query('Numbers == 2')
                   ).mark_text(dx=35, dy=-85, fontSize=28, fontWeight='normal', color='#ffbe94'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))
midwest = alt.Chart(region_text_df.query('Numbers == 3')
                   ).mark_text(dx=51, dy=23, fontSize=28, fontWeight='normal', color='#c86300'
                              ).encode(x=alt.X("Year", axis=None), y=alt.Y("Numbers"), text=alt.Text("Text"))

line = alt.Chart(pd.DataFrame({'x':[1994], 'y':[7], 'text':['_'*74]})).mark_text(dx=65, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')
y_labels_df = pd.DataFrame({'x':[1989], 'y':[7], 'text':['7 times income']})
y_label = alt.Chart(y_labels_df.query('x == 1989')).mark_text(dx=-247, dy=0, font='lato', fontSize=22, opacity=.7).encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')

region_chart = reg_chart + west + northeast + south + midwest + line + y_label
region_chart = region_chart.configure_title(fontSize=28)"""

alt.LayerChart(...)

In [80]:
region_chart

alt.LayerChart(...)

In [79]:
#region_chart.save('region_hir_8918.png', scale_factor=3)

---

### New Home Sales (by region)

In [72]:
sold_new_homes.columns = ['Year', 'US', 'Northeast', 'Midwest', 'South', 'West']
sold_new_homes = sold_new_homes[['Year', 'US', 'Midwest', 'Northeast', 'South', 'West']]

In [143]:
# Change in new home sales in each region since low in 2011
pct_diffs = []
for col in sold_new_homes.columns[2:]:
    col11 = sold_new_homes[col][sold_new_homes.Year == 2011].iloc[0]
    col19 = sold_new_homes[col][sold_new_homes.Year == 2019].iloc[0]
    pct_diffs.append(round((col19 - col11) / col11 *100, 4))

In [144]:
pct_diffs

[55.5556, 42.8571, 137.5, 152.7778]

Manipulating df structure for plotting:

In [73]:
vals = []
for col in sold_new_homes.columns[2:]:
    vals += sold_new_homes[col].values.tolist()

In [74]:
cols = list(sold_new_homes.columns[2:])*len(sold_new_homes)
cols.sort()

In [75]:
sold_df = pd.DataFrame({'Year':list(range(1963, 2020))*4, 'Region':cols, 'Sold':vals})

In [185]:
chart = alt.Chart(sold_df[sold_df.Year > 1995], title='US New Home Sales'
                 ).mark_line(size=7, strokeCap='round', interpolate='basis'
                            ).encode(x=alt.X('Year:O', 
                                             axis=alt.Axis(values=[2000,2010,2019], 
                                                           ticks=True, tickSize=10,
                                                           labelFontSize=24, labelOpacity=.7)), 
                                     y=alt.Y('Sold', title='Regional comparison',
                                             axis=alt.Axis(values=list(range(100, 700, 100)), 
                                                           titleX=-40, titleY=-40, titleFontSize=26, titleOpacity=.7,
                                                           labelFontSize=24, labelPadding=40, labelOpacity=.7), 
                                             scale=alt.Scale(domain=[0,700])), 
                                     color=alt.Color('Region', title='', 
                                                     scale=alt.Scale(domain=['South', 'West', 'Midwest', 'Northeast'], 
                                                                     range=['#ffd5b0', '#004d66', '#c86300', '#00a58a'])))

line = alt.Chart(pd.DataFrame({'x':[1996], 'y':[700], 'text':['_'*74]})).mark_text(dx=350, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')
y_labels_df = pd.DataFrame({'x':[1996], 'y':[700], 'text':['700k new homes']})
y_label = alt.Chart(y_labels_df.query('x == 1996')).mark_text(dx=37, dy=0, font='lato', fontSize=24, opacity=.7).encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')

chart = chart + line + y_label
chart = chart.configure_title(fontSize=30).configure_legend(orient='none', legendX=605, legendY=260, 
                                    labelFont='lato', labelFontSize=30, labelOpacity=.7,
                                    symbolSize=200, symbolStrokeWidth=7)

In [186]:
chart

alt.LayerChart(...)

In [187]:
chart.save('reg_home_sales0019.png', scale_factor=3)

---

### CA Counties Med. HIR

Renters:

In [4]:
# Relied on 5-yr for 08-16 and supplemented 1-yr for 17 & 18 since county data is succeptible to greater change between years (due to smaller sample data pools)
ca0818 = pd.concat([county5[(county5.FIPS > 6000) & (county5.FIPS < 7000) & (county5.FIPS.isin(county1.FIPS.unique()))], 
                    county1[(county1.Year > 2016) & (county1.FIPS > 6000) & (county1.FIPS < 7000)]])
la = ca0818[ca0818.FIPS == 6037]
ca_no_la = ca0818[ca0818.FIPS != 6037]

In [5]:
ca_rent_per = alt.Chart(ca_no_la, title='Renter Income Devoted to Housing'
                       ).mark_line(opacity=.6, interpolate='basis'
                                  ).encode(x=alt.X('Year:O', 
                                                   axis=alt.Axis(ticks=True, tickSize=10, values=[2008, 2018],
                                                                 labelFontSize=18, labelOpacity=.7)), 
                                           y=alt.Y('Rent_Share_Inc', 
                                                    title='LA relative to all other CA counties', 
                                                    axis=alt.Axis(titleFontSize=20, titleX=-30, titleY=-30, titleOpacity=.7,
                                                                  labelFontSize=18, labelPadding=30, labelOpacity=.7), 
                                                    scale=alt.Scale(domain=[10,50])), 
                                            color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))

la_rent_per = alt.Chart(la).mark_line(color='#fbcb42', size=6, strokeCap='round', interpolate='basis').encode(x=alt.X('Year:O', axis=alt.Axis(labelFontSize=18, values=[2008, 2018])), 
                                                                      y=alt.Y('Rent_Share_Inc', 
                                                                              axis=alt.Axis(values=list(range(15, 50, 5))), 
                                                                              scale=alt.Scale(domain=[10, 50])))
y_labels_df = pd.DataFrame({'x':[2008,2009], 'y':[50,50], 'text':['50% of income','_'*79]})
y_label = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=2, dy=0, font='lato', fontSize=18, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(y_labels_df.query('x == 2009')).mark_text(dx=265, dy=-4, fontSize=11.5, fontWeight='bold', opacity=.09).encode(x='x:O', y='y', text='text')
rent_per_inc = ca_rent_per + la_rent_per + y_label + line

In [6]:
rent_per_inc

alt.LayerChart(...)

In [7]:
rent_per_inc.save('ca_renter_inc_per_housing.png', scale_factor=3)

Homeowners:

In [8]:
ca_mort_per = alt.Chart(ca_no_la, title='Homeowner Income Devoted to Housing'
                       ).mark_line(opacity=.6, interpolate='basis'
                                  ).encode(x=alt.X('Year:O', 
                                                   axis=alt.Axis(ticks=True, tickSize=10, values=[2008, 2018],
                                                                 labelFontSize=18, labelOpacity=.7)), 
                                           y=alt.Y('Mort_Share_Inc', 
                                                    title='LA relative to all other CA counties', 
                                                    axis=alt.Axis(titleFontSize=20, titleX=-30, titleY=-30, titleOpacity=.7,
                                                                  labelFontSize=18, labelPadding=30, labelOpacity=.7), 
                                                    scale=alt.Scale(domain=[10,50])), 
                                           color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))
la_mort_per = alt.Chart(la).mark_line(color='#fbcb42', size=6, strokeCap='round', interpolate='basis').encode(x=alt.X('Year:O', axis=alt.Axis(labelFontSize=18, values=[2008, 2018])), 
                                                                                         y=alt.Y('Mort_Share_Inc', 
                                                                              axis=alt.Axis(values=list(range(15, 50, 5))), 
                                                                              scale=alt.Scale(domain=[10, 50])))

y_labels_df = pd.DataFrame({'x':[2008,2009], 'y':[50,50], 'text':['50% of income','_'*79]})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=2, dy=0, font='lato', fontSize=18, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(y_labels_df.query('x == 2009')).mark_text(dx=265, dy=-4, fontSize=11.5, fontWeight='bold', opacity=.09).encode(x='x:O', y='y', text='text')

mort_per_inc = ca_mort_per + la_mort_per + y_labels + line

In [9]:
mort_per_inc

alt.LayerChart(...)

In [10]:
mort_per_inc.save('ca_homeowner_inc_per_housing.png', scale_factor=3)

In [163]:
ca0818[['Year', 'County', 'Mort_Share_Inc_whole']][ca0818.Year == 2016].sort_values(['Year','Mort_Share_Inc_whole'], ascending=[True, False]).head(10)

,Year,County,Mort_Share_Inc_whole
25186,2016,"Los Angeles County, California",24.6
25200,2016,"Riverside County, California",23.9
25204,2016,"San Diego County, California",23.8
25211,2016,"Santa Cruz County, California",23.6
25223,2016,"Ventura County, California",23.6
25188,2016,"Marin County, California",23.2
25196,2016,"Nevada County, California",23.2
25197,2016,"Orange County, California",23.1
25207,2016,"San Luis Obispo County, California",22.7
25216,2016,"Sonoma County, California",22.5


---

### HIR County Distribution

2018 Distribution:

In [306]:
plot_df = county_hir.copy()
plot_df['pct_bin'] = plot_df.HIR.round(1)

In [307]:
yrs_dict = {}
for year in plot_df.Year.unique():
    yr_dict = {}    
    yr_df = plot_df[plot_df.Year == year]
    for val in yr_df.pct_bin.unique():
        yr_dict[val] = len(yr_df[yr_df.pct_bin == val])
    yrs_dict[year] = yr_dict

In [19]:
# Identifying max to manipulate chart so y-axis is consistent for all charts
for year in yrs_dict:
    print(year, max(yrs_dict[year].values()))

1989 102
1990 109
1991 116
1992 134
1993 161
1994 155
1995 164
1996 165
1997 167
1998 169
1999 193
2000 166
2001 171
2002 174
2003 153
2004 150
2005 150
2006 133
2007 133
2008 139
2009 152
2010 188
2011 176
2012 186
2013 195
2014 204
2015 191
2016 194
2017 196
2018 180


In [20]:
# Identifying max to manipulate chart so y-axis is consistent for all charts
for year in yrs_dict:
    print(year, max(yrs_dict[year].keys()))

1989 14.3
1990 12.8
1991 11.6
1992 10.4
1993 9.4
1994 8.3
1995 8.2
1996 7.9
1997 7.8
1998 8.9
1999 9.2
2000 10.6
2001 11.4
2002 11.7
2003 13.0
2004 13.5
2005 15.6
2006 17.6
2007 17.8
2008 15.4
2009 15.0
2010 13.7
2011 13.0
2012 13.0
2013 13.0
2014 12.4
2015 13.1
2016 15.2
2017 13.6
2018 13.4


In [357]:
counts_df = pd.DataFrame(yrs_dict[2018].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
counts_df = counts_df[(counts_df.Bins < 7) & ~((counts_df.Bins > 6) & (counts_df.Counts > 3))]
text_line_df = pd.DataFrame({'x1':[2.75], 'x2':[5.2], 'x3':[5.7], 'x4':[5.6], 'x5':[5.78], 'y1':[179], 'y2':[179], 'y3':[160], 'y4':[150], 'y5':[140], 'Text1':['In 180 counties, median home value'], 'Text2':['was 2.6 times median household income']})
text_df = pd.DataFrame({'Bins':[1,7], 'Counts':[200,200], 'Text':[1989, 2018]})

# Charts
chart = alt.Chart(counts_df, title='Home Value Relative to Annual Income (2018)'
                 ).mark_bar(size=7.5, color='#4c92d8', cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                 ).encode(x=alt.X('Bins', title='Home/Income Multiple',
                                  axis=alt.Axis(values=list(range(8)), titleY=30, titleFontSize=20, titleOpacity=.7, labelOpacity=0)), 
                          y=alt.Y('Counts:Q', title='Distribution of median values for all counties',
                                  axis=alt.Axis(values=list(range(40,200,40)), titleX=-30, titleY=0, titleFontSize=20, titleOpacity=.7,
                                                labelPadding=30, labelFontSize=18, labelOpacity=.7), 
                                                scale=alt.Scale(domain=[0,210])))
x_vals = alt.Chart(pd.DataFrame({'x':list(range(1,8)), 'y':[0]*7, 'text':list(range(1,8))})
                  ).mark_text(dx=0, dy=20, fontSize=18, opacity=.7
                             ).encode(x=alt.X('x'), y=alt.Y('y', axis=None), text='text')

y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-21, dy=0, fontSize=18, font='lato', opacity=.7).encode(x='x', y='y', text='text')
line = alt.Chart(pd.DataFrame({'x1':[1.32], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.15).encode(x='x1', x2='x2', y='y1')
line1 = alt.Chart(text_line_df).mark_rule(size=2, color='#999999', strokeDash=[1,2.5]).encode(x='x1', x2='x2', y='y1', y2='y2')
line2 = alt.Chart(text_line_df).mark_rule(size=2, color='#999999', strokeDash=[1,2.5]).encode(x='x2', x2='x3', y='y2', y2='y3')
text1 = alt.Chart(text_line_df.query('x1 == 2.75')).mark_text(dx=0, dy=0, fontSize=16, font='lato', opacity=.7).encode(x='x4', y='y4', text='Text1')
text2 = alt.Chart(text_line_df.query('x1 == 2.75')).mark_text(dx=0, dy=0, fontSize=16, font='lato', opacity=.7).encode(x='x5', y='y5', text='Text2')

plot = chart + x_vals + y_text + line + line1 + line2 + text1 + text2

In [359]:
plot

alt.LayerChart(...)

In [410]:
plot.save('hir_county_dist_2018.png', scale_factor=3)

---

#### 1989-1999 Distribution

In [10]:
years = range(1989,2000)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 7]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,7,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='Home Value Relative to Annual Income (\'89-\'99)'
                   ).mark_bar(color='#4c92d8', size=8, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home/Income Multiple', 
                                              axis=alt.Axis(values=list(range(1,8)), 
                                                            titleOpacity=.7, titleFontSize=20, titleY=35, 
                                                            labelFontSize=18, labelOpacity=.7)), 
                                      y=alt.Y('Counts:Q', scale=alt.Scale(domain=[0,210]), 
                                              axis=alt.Axis(titleFontSize=20, titleX=-35, 
                                                            labelOpacity=.7, labelPadding=35, labelFontSize=18, 
                                                            values=list(range(40,200,40)))))
    y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-72, dy=0, fontSize=18, font='lato', fontWeight='normal', opacity=.09).encode(x='x:O', y='y', text='text')
    line = alt.Chart(pd.DataFrame({'x1':[.8], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.01).encode(x='x1:O', x2='x2', y='y1')
    chart = var + y_text + line
    variables.append(chart)
    counter += 1

combined8999 = chart   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart (b/c it's already assigned to 'test')
    combined8999 += var

combined8999

alt.LayerChart(...)

In [11]:
combined8999.save('combined8999.png', scale_factor=3)

---

#### 2000-2010 Distribution

In [12]:
years = range(2000,2010)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 7]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,7,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='Home Value Relative to Annual Income (\'00-\'09)'
                   ).mark_bar(color='#4c92d8', size=8, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home/Income Multiple', 
                                              axis=alt.Axis(values=list(range(1,8)), 
                                                            titleFontSize=20, titleOpacity=.7, titleY=35, 
                                                            labelFontSize=18, labelOpacity=.7)), 
                                      y=alt.Y('Counts:Q', scale=alt.Scale(domain=[0,210]), 
                                              axis=alt.Axis(titleFontSize=20, titleX=-35, 
                                                            labelPadding=35, labelFontSize=18, labelOpacity=.7, 
                                                            values=list(range(40,200,40)))))
    y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-72, dy=0, fontSize=18, font='lato', fontWeight='normal', opacity=.1).encode(x='x:O', y='y', text='text')
    line = alt.Chart(pd.DataFrame({'x1':[.8], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.01).encode(x='x1:O', x2='x2', y='y1')
    chart = var + y_text + line
    variables.append(chart)
    counter += 1

combined0009 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart
    combined0009 += var

combined0009

alt.LayerChart(...)

In [13]:
combined0009.save('combined0009.png', scale_factor=3)

---

#### 2010-2018 Distribution

In [16]:
years = range(2010,2019)
variables = []
counter = 0
for year in years:
    df = pd.DataFrame(yrs_dict[year].items(), columns=['Bins', 'Counts']).sort_values(['Bins'])
    df = df[df.Bins <= 7]  # Removing outliers (to ensure consistent x axis values)
    add_vals = []
    for val in np.arange(0,7,.1):
        if round(val,1) not in df.Bins.values:
            add_vals.append(round(val,1))
    missing_vals = pd.DataFrame({'Bins':add_vals, 'Counts':list(np.full(len(add_vals),0))})
    df = pd.concat([df, missing_vals]).sort_values('Bins')
    
    var = alt.Chart(df, title='Home Value Relative to Annual Income (\'10-\'18)'
                   ).mark_bar(color='#4c92d8', size=8, opacity=.3, cornerRadiusTopLeft=1, cornerRadiusTopRight=1
                             ).encode(x=alt.X('Bins:O', title='Home/Income Multiple', 
                                              axis=alt.Axis(values=list(range(1,8)), 
                                                            titleFontSize=20, titleOpacity=.7, titleY=35,
                                                            labelFontSize=18, labelOpacity=.7)), 
                                      y=alt.Y('Counts:Q',
                                              scale=alt.Scale(domain=[0,210]), 
                                              axis=alt.Axis(titleFontSize=20, titleX=-35, labelPadding=35, labelFontSize=18, labelOpacity=.7, values=list(range(40,200,40)))))
    y_text = alt.Chart(pd.DataFrame({'x':[1], 'y':[200], 'text':['200 counties']})).mark_text(dx=-72, dy=0, fontSize=18, font='lato', fontWeight='normal', opacity=.12).encode(x='x:O', y='y', text='text')
    line = alt.Chart(pd.DataFrame({'x1':[.8], 'x2':[7], 'y1':[200]})).mark_rule(opacity=.01).encode(x='x1:O', x2='x2', y='y1')
    chart = var + y_text + line
    variables.append(chart)    
    counter += 1

combined1018 = var   # Last chart created (must assign altair chart object to variable to enable appending)
for var in variables[:-1]:  # Excluding last chart
    combined1018 += var

combined1018

alt.LayerChart(...)

In [15]:
combined1018.save('combined1018.png', scale_factor=3)

---

### HIR by County Household Count:

In [140]:
hir_pop = alt.Chart(hh_count_hir, title='Home Value Relative to Annual Income').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[1990, 2000, 2010, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('HIR', title='Counties grouped by number of households', 
        scale=alt.Scale(domain=[0,8]), 
        axis=alt.Axis(titleX=-15, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=15,
                      values=list(range(2,8,2)), format='a')),
color=alt.Color('HH_Count', scale=alt.Scale(domain=list(hh_count_hir.HH_Count.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)')
)

#x_vals_df = pd.DataFrame({'x':list(range(1990, 2019,4)), 'y':[-1]*8, 'text':['\'' + str(y)[-2:] for y in range(1990,2019,4)]})
#x_vals = alt.Chart(x_vals_df).mark_text(dx=0, dy=-30, opacity=.7, font='lato', fontSize=18).encode(x='x:O', y='y', text='text')

y_text_df = pd.DataFrame({'x':[1989], 'y':[8], 'text':['8 times income']})
y_text = alt.Chart(y_text_df).mark_text(dx=35, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x1':[1994], 'x2':[2018], 'y1':[8]})).mark_rule(opacity=.15).encode(x='x1:O', x2='x2:O', y='y1')

hir_hh_chart = (hir_pop + y_text + line).configure_legend(orient='none', legendX=610, legendY=87, titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)
hir_hh_chart

alt.LayerChart(...)

In [141]:
hir_hh_chart.save('hir_hh_count.png', scale_factor=3)

---

### Housing % of Income (by county size)

In [142]:
hh_count_df = pd.concat([hh_count_per_inc5, hh_count_per_inc1[hh_count_per_inc1.Year > 2016]])

*Adjusting 2017 & 2018 values for '10-20' & 'Less than 10' cols (since 1-yr data included few counties of low populations, resulting in distorted figures)

In [143]:
hh_count_df_rents = hh_count_df[hh_count_df.Type == 'Rents']
hh_count_df_morts = hh_count_df[hh_count_df.Type == 'Morts']
hh_count_df_rents['pct_chg'] = hh_count_df_rents['20 to 50'].pct_change()
hh_count_df_morts['pct_chg'] = hh_count_df_morts['20 to 50'].pct_change()

C:\Users\Mahmoud\Miniconda3\envs\data_analysis\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Mahmoud\Miniconda3\envs\data_analysis\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [144]:
for df in [hh_count_df_rents, hh_count_df_morts]:
    for col in ['Less than 10', '10 to 20']:
        for year in [2017, 2018]:
            df.loc[df.Year == year, col] = round(df[col][df.Year == year-1].iloc[0] + (df[col][df.Year == year-1].iloc[0]) * df.pct_chg[df.Year == year], 3)
            
hh_count_df = pd.concat([hh_count_df_rents, hh_count_df_morts]).sort_values(['Year', 'Type'], ascending=[True, False]).drop('pct_chg', axis=1)

C:\Users\Mahmoud\Miniconda3\envs\data_analysis\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [145]:
# Manipulating data to plot multiple lines on chart
rent = hh_count_df[hh_count_df.Type == 'Rents']
mort = hh_count_df[hh_count_df.Type == 'Morts']
df_dict = {}
for type_df in [rent, mort]:
    vals = []
    for col in type_df.columns[:-2]:
        vals += type_df[col].values.tolist()
    df = pd.DataFrame({'Year':list(range(2008, 2019))*7, 'Home_Per_Inc':[round(i,3)*100 for i in vals]})
    house_types = []
    for typ in type_df.columns[:-2]:
        house_types += [typ]*11
    df['Type'] = house_types
    df_dict[type_df.Type.iloc[0]] = df

In [152]:
hh_count_mort_df = pd.DataFrame(df_dict['Morts'])
hh_count_mort = alt.Chart(hh_count_mort_df, title='Homeowner Income Devoted to Housing'
                    ).mark_line(size=6, strokeCap='round', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, labelFontSize=18, labelOpacity=.7)), 
                                        y=alt.Y('Home_Per_Inc', title='Counties grouped by number of households',
                                               axis=alt.Axis(values=list(range(15, 40, 5)), 
                                                             titleX=-25,
                                                             titleY=-35,
                                                             titleFontSize=20,
                                                             titleOpacity=.7,
                                                             labelPadding=25, 
                                                             labelOpacity=.7,
                                                             labelFontSize=18), 
                                               scale=alt.Scale(domain=[10, 40])),
                                        color=alt.Color('Type', legend=None, scale=alt.Scale(domain=list(hh_count_mort_df.Type.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)'))

y_labels_df = pd.DataFrame({'x':[2008, 2009], 'y':[40, 40], 'text':['40% of income', '_'*83]})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=6, dy=0, font='lato', fontSize=18, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(y_labels_df.query('x == 2009')).mark_text(dx=267, dy=-5, fontWeight='bold', opacity=.12).encode(x='x:O', y='y', text='text')

hh_count_mort_chart = hh_count_mort + y_labels + line

In [153]:
hh_count_mort_chart

alt.LayerChart(...)

In [154]:
hh_count_rent_df = pd.DataFrame(df_dict['Rents'])
hh_count_rent = alt.Chart(hh_count_rent_df, title='Renter Income Devoted to Housing'
                    ).mark_line(size=6, strokeCap='round', interpolate='basis'
                               ).encode(x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, labelFontSize=18, labelOpacity=.7)), 
                                        y=alt.Y('Home_Per_Inc', title='Counties grouped by number of households',
                                               axis=alt.Axis(values=list(range(15, 40, 5)), 
                                                             titleX=-25,
                                                             titleY=-35,
                                                             titleFontSize=20,
                                                             titleOpacity=.7,
                                                             labelPadding=25, 
                                                             labelOpacity=.7,
                                                             labelFontSize=18), 
                                               scale=alt.Scale(domain=[10, 40])),
                                        color=alt.Color('Type', legend=None, scale=alt.Scale(domain=list(hh_count_rent_df.Type.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Households (thousands)'))

y_labels_df = pd.DataFrame({'x':[2008, 2009], 'y':[40, 40], 'text':['40% of income', '_'*83]})
y_labels = alt.Chart(y_labels_df.query('x == 2008')).mark_text(dx=6, dy=0, font='lato', fontSize=18, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(y_labels_df.query('x == 2009')).mark_text(dx=267, dy=-5, fontWeight='bold', opacity=.12).encode(x='x:O', y='y', text='text')

hh_count_rent_chart = hh_count_rent + y_labels + line

In [155]:
hh_count_rent_chart

alt.LayerChart(...)

In [156]:
hh_count_mort_chart.save('hh_count_mort_chart.png', scale_factor=3)

In [157]:
hh_count_rent_chart.save('hh_count_rent_chart.png', scale_factor=3)

---

### Housing % of Income (by population density)

In [605]:
mort_pop_den_df = pd.concat([mort_share_inc_pop_den5, mort_share_inc_pop_den1[mort_share_inc_pop_den1.Year > 2016]]).sort_values(['Home_Units', 'Year']).reset_index(drop=True)

*Adjusting 2017 & 2018 values for some cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [606]:
mort_pop_den_df['pct_chg'] = mort_pop_den_df['Mort_Share_Inc'].pct_change()

In [607]:
for year in [2017, 2018]:
    yr_df = mort_pop_den_df[mort_pop_den_df.Year == year]
    for typ in ['100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
        index = yr_df[yr_df.Home_Units == typ].index[0]
        mort_pop_den_df.loc[index, 'Mort_Share_Inc'] = round(mort_pop_den_df['Mort_Share_Inc'][(mort_pop_den_df.Year == year-1) & (mort_pop_den_df.Home_Units == typ)].iloc[0] + mort_pop_den_df['Mort_Share_Inc'][(mort_pop_den_df.Year == year-1) & (mort_pop_den_df.Home_Units == typ)].iloc[0] * mort_pop_den_df.pct_chg[(mort_pop_den_df.Year == year) & (mort_pop_den_df.Home_Units == 'More than 1000')].iloc[0], 3)

In [608]:
dic = {}
for t in ['More than 1000', '250 to 1000', '100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
    dic[t] = t.split()[-1]

In [609]:
mort_pop_den_df['Order'] = mort_pop_den_df.Home_Units.map(dic).astype(int)
mort_pop_den_df = mort_pop_den_df.sort_values(['Order', 'Home_Units'], ascending=[False, False])

In [610]:
mort_pop_den = alt.Chart(mort_pop_den_df, title='Homeowner Income Devoted to Housing').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('Mort_Share_Inc', title='Counties grouped by population density', 
        scale=alt.Scale(domain=[10,40]),
        axis=alt.Axis(values=list(range(15,40,5)),
                      titleX=-25, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=25)),
color=alt.Color('Home_Units', scale=alt.Scale(domain=list(mort_pop_den_df.Home_Units.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Residents / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_text = alt.Chart(y_text_df).mark_text(dx=6, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[40], 'text':['_'*79]})).mark_text(dx=320, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


mort_pop_den_chart = (mort_pop_den + y_text + line).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

In [611]:
mort_pop_den_chart

alt.LayerChart(...)

In [612]:
mort_pop_den_chart.save('mort_pop_den.png', scale_factor=3)

---

In [596]:
rent_pop_den_df = pd.concat([rent_share_inc_pop_den5, rent_share_inc_pop_den1[rent_share_inc_pop_den1.Year > 2016]]).sort_values(['Home_Units', 'Year']).reset_index(drop=True)

*Adjusting 2017 & 2018 values for some cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [597]:
rent_pop_den_df['pct_chg'] = rent_pop_den_df['Rent_Share_Inc'].pct_change()

In [598]:
for year in [2017, 2018]:
    yr_df = rent_pop_den_df[rent_pop_den_df.Year == year]
    for typ in ['100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
        index = yr_df[yr_df.Home_Units == typ].index[0]
        rent_pop_den_df.loc[index, 'Rent_Share_Inc'] = round(rent_pop_den_df['Rent_Share_Inc'][(rent_pop_den_df.Year == year-1) & (rent_pop_den_df.Home_Units == typ)].iloc[0] + rent_pop_den_df['Rent_Share_Inc'][(rent_pop_den_df.Year == year-1) & (rent_pop_den_df.Home_Units == typ)].iloc[0] * rent_pop_den_df.pct_chg[(rent_pop_den_df.Year == year) & (rent_pop_den_df.Home_Units == 'More than 1000')].iloc[0], 3)

In [599]:
dic = {}
for t in ['More than 1000', '250 to 1000', '100 to 250', '50 to 100', '25 to 50', '10 to 25', 'Less than 10']:
    dic[t] = t.split()[-1]

In [600]:
rent_pop_den_df['Order'] = rent_pop_den_df.Home_Units.map(dic).astype(int)

In [601]:
rent_pop_den_df = rent_pop_den_df.sort_values(['Order', 'Home_Units'], ascending=[False, False])

In [602]:
rent_pop_den = alt.Chart(rent_pop_den_df, title='Renter Income Devoted to Housing').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('Rent_Share_Inc', title='Counties grouped by population density', 
        scale=alt.Scale(domain=[10,40]),
        axis=alt.Axis(values=list(range(15,40,5)),
                      titleX=-25, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=25)),
color=alt.Color('Home_Units', scale=alt.Scale(domain=list(rent_pop_den_df.Home_Units.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#f8f6e9','#d1e5f0','#67a9cf','#2166ac']), title='Residents / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_text = alt.Chart(y_text_df).mark_text(dx=6, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[40], 'text':['_'*79]})).mark_text(dx=320, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


rent_pop_den_chart = (rent_pop_den + y_text + line).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

In [603]:
rent_pop_den_chart

alt.LayerChart(...)

In [604]:
rent_pop_den_chart.save('rent_pop_den.png', scale_factor=3)

---

### Housing % of Income (by housing density)

In [562]:
mort_hous_den_df = pd.concat([mort_share_inc_hous_den5, mort_share_inc_hous_den1[mort_share_inc_hous_den1.Year > 2016]]).sort_values(['Home_Units', 'Year']).reset_index(drop=True)

*Adjusting 2017 & 2018 values for most cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [563]:
mort_hous_den_df['pct_chg'] = mort_hous_den_df['Mort_Share_Inc'].pct_change()

In [569]:
for year in [2017, 2018]:
    yr_df = mort_hous_den_df[mort_hous_den_df.Year == year]
    for typ in ['30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
        index = yr_df[yr_df.Home_Units == typ].index[0]
        mort_hous_den_df.loc[index, 'Mort_Share_Inc'] = round(mort_hous_den_df['Mort_Share_Inc'][(mort_hous_den_df.Year == year-1) & (mort_hous_den_df.Home_Units == typ)].iloc[0] + mort_hous_den_df['Mort_Share_Inc'][(mort_hous_den_df.Year == year-1) & (mort_hous_den_df.Home_Units == typ)].iloc[0] * mort_hous_den_df.pct_chg[(mort_hous_den_df.Year == year) & (mort_hous_den_df.Home_Units == 'More than 75')].iloc[0], 3)

In [571]:
dic = {}
for t in ['More than 75', '30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
    dic[t] = t.split()[-1]

In [572]:
mort_hous_den_df['Order'] = mort_hous_den_df.Home_Units.map(dic).astype(int)
mort_hous_den_df = mort_hous_den_df.sort_values(['Order', 'Home_Units'], ascending=[False, False])

In [593]:
mort_hous_den = alt.Chart(mort_hous_den_df, title='Homeowner Income Devoted to Housing').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('Mort_Share_Inc', title='Counties grouped by housing density', 
        scale=alt.Scale(domain=[10,40]),
        axis=alt.Axis(values=list(range(15,40,5)),
                      titleX=-25, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=25)),
color=alt.Color('Home_Units', scale=alt.Scale(domain=list(mort_hous_den_df.Home_Units.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#d1e5f0','#67a9cf','#2166ac']), title='Housing Units / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_text = alt.Chart(y_text_df).mark_text(dx=6, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[40], 'text':['_'*79]})).mark_text(dx=320, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


mort_hous_den_chart = (mort_hous_den + y_text + line).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

In [594]:
mort_hous_den_chart

alt.LayerChart(...)

In [595]:
mort_hous_den_chart.save('mort_hous_den.png', scale_factor=3)

---

In [579]:
rent_hous_den_df = pd.concat([rent_share_inc_hous_den5, rent_share_inc_hous_den1[rent_share_inc_hous_den1.Year > 2016]]).sort_values(['Home_Units', 'Year']).reset_index(drop=True)

*Adjusting 2017 & 2018 values for most cols (since 1-yr data included few counties of low pop densities, resulting in distorted figures)

In [581]:
rent_hous_den_df['pct_chg'] = rent_hous_den_df['Rent_Share_Inc'].pct_change()

In [582]:
for year in [2017, 2018]:
    yr_df = rent_hous_den_df[rent_hous_den_df.Year == year]
    for typ in ['30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
        index = yr_df[yr_df.Home_Units == typ].index[0]
        rent_hous_den_df.loc[index, 'Rent_Share_Inc'] = round(rent_hous_den_df['Rent_Share_Inc'][(rent_hous_den_df.Year == year-1) & (rent_hous_den_df.Home_Units == typ)].iloc[0] + rent_hous_den_df['Rent_Share_Inc'][(rent_hous_den_df.Year == year-1) & (rent_hous_den_df.Home_Units == typ)].iloc[0] * rent_hous_den_df.pct_chg[(rent_hous_den_df.Year == year) & (rent_hous_den_df.Home_Units == 'More than 75')].iloc[0], 3)

In [583]:
dic = {}
for t in ['More than 75', '30 to 75', '20 to 30', '10 to 20', '5 to 10', 'Less than 5']:
    dic[t] = t.split()[-1]

In [584]:
rent_hous_den_df['Order'] = rent_hous_den_df.Home_Units.map(dic).astype(int)
rent_hous_den_df = rent_hous_den_df.sort_values(['Order', 'Home_Units'], ascending=[False, False])

In [589]:
rent_hous_den = alt.Chart(rent_hous_den_df, title='Renter Income Devoted to Housing').mark_line(size=6, opacity=.85, strokeCap='round', interpolate='basis').encode(
x=alt.X('Year:O', axis=alt.Axis(values=[2008, 2018], ticks=True, tickSize=10, titleFontSize=20, labelFontSize=18, labelOpacity=.7)),
y=alt.Y('Rent_Share_Inc', title='Counties grouped by housing density', 
        scale=alt.Scale(domain=[10,40]),
        axis=alt.Axis(values=list(range(15,40,5)),
                      titleX=-25, titleY=-30, titleFontSize=20, titleOpacity=.7,
                      labelFontSize=18, labelOpacity=.7, labelPadding=25)),
color=alt.Color('Home_Units', scale=alt.Scale(domain=list(rent_hous_den_df.Home_Units.unique()), 
                                             range=['#b2182b','#ef8a62','#fddbc7','#d1e5f0','#67a9cf','#2166ac']), title='Housing Units / sq. mile')
)

y_text_df = pd.DataFrame({'x':[2008], 'y':[40], 'text':['40% of income']})
y_text = alt.Chart(y_text_df).mark_text(dx=6, dy=0, fontSize=18, font='lato', opacity=.7).encode(x=alt.X('x:O'), y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[2008], 'y':[40], 'text':['_'*79]})).mark_text(dx=320, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')


rent_hous_den_chart = (rent_hous_den + y_text + line).configure_legend(titleFont='Lato', titleFontSize=20, titleOpacity=.85, titleLimit=250, labelFont='Lato', labelFontSize=20, labelOpacity=.85, symbolSize=300, symbolStrokeWidth=6)

In [590]:
rent_hous_den_chart

alt.LayerChart(...)

In [592]:
rent_hous_den_chart.save('rent_hous_den.png', scale_factor=3)

---

### LA vs. Other HIR 

In [51]:
# Ranking CA Counties by Avg. HIR
ca_counties = county_hir[(county_hir.FIPS > 6000) & (county_hir.FIPS < 7000)]
fips_dict = {}
for fips in ca_counties.FIPS.unique():
    df = ca_counties[ca_counties.FIPS == fips]
    fips_dict[fips] = df.HIR.mean().round(3)
    
avg_hirs = sorted(fips_dict.items(), key=lambda x:x[1], reverse=True)

In [52]:
avg_hirs[:8]  # LA has the 6th highest avg. HIR in CA

[(6075, 9.089),
 (6087, 8.465),
 (6041, 8.078),
 (6081, 7.745),
 (6083, 7.408),
 (6037, 7.283),
 (6079, 7.277),
 (6045, 7.181)]

In [53]:
la_df = county_hir[county_hir.FIPS == 6037]

In [54]:
ca_df = county_hir[(county_hir.FIPS != 6037) & (county_hir.FIPS > 6000) & (county_hir.FIPS < 7000)]

In [55]:
text_df = la_df.copy()
text_df['Text'] = list(np.full(len(text_df)-5, np.nan)) + ['CA', 'Counties', 'National', 'California', 'LA County']

In [70]:
ca_counties = alt.Chart(ca_df, title='Home Value Relative to Annual Income').mark_line(opacity=.6, interpolate='basis').encode(
    x=alt.X('Year:O', axis=alt.Axis(values=[1990, 2000, 2010, 2018],
                                    ticks=True, tickSize=10,
                                    labelFontSize=16, labelOpacity=.7)), 
    y=alt.Y('HIR', title='Comparing LA county to CA median, nat\'l median and CA\'s other counties', 
            scale=alt.Scale(domain=[0,14]), 
            axis=alt.Axis(values=list(range(2,14,2)), 
                          titleFontSize=18, titleOpacity=.7, titleX=-20, titleY=-30,
                          labelFontSize=16, labelOpacity=.7, labelPadding=20)),
    color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))

nat_med = alt.Chart(nat_hir).mark_line(color='#9a9a9a', size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

ca_med = alt.Chart(ca_hir).mark_line(size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

la = alt.Chart(la_df).mark_line(color='#fbcb42', size=5, strokeCap='round', interpolate='basis').encode(
    x=alt.X('Year:O'), 
    y=alt.Y('HIR'))

CA_text = (
    alt.Chart(text_df.query('Year == 2014'))
    .mark_text(dx=105, dy=60, fontSize=20, fontWeight='normal', color='#afafaf')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

cou_text = (
    alt.Chart(text_df.query('Year == 2015'))
    .mark_text(dx=111, dy=83, fontSize=20, fontWeight='normal', color='#afafaf')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)
nat_text = (
    alt.Chart(text_df.query('Year == 2016'))
    .mark_text(dx=94, dy=141, fontSize=20, fontWeight='bold', color='#858585')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

ca_text = (
    alt.Chart(text_df.query('Year == 2017'))
    .mark_text(dx=77, dy=28, fontSize=20, fontWeight='bold', color='#15607a')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

la_text = (
    alt.Chart(text_df.query('Year == 2018'))
    .mark_text(dx=60, dy=-3, fontSize=20, fontWeight='bold', color='#ffca00')
    .encode(x=alt.X("Year:O", axis=None), y=alt.Y("HIR"), text=alt.Text("Text"))
)

y_label = alt.Chart(pd.DataFrame({'x':[1989], 'y':[14], 'text':['14 times income']}).query('x == 1989')).mark_text(dx=27, font='lato', fontSize=16, opacity=.7).encode(x='x:O', y='y', text='text')
line = alt.Chart(pd.DataFrame({'x':[1992], 'y':[14], 'text':['_'*79]}).query('x == 1992')).mark_text(dx=279, dy=-4, fontSize=11.5, fontWeight='bold', opacity=.09).encode(x='x:O', y='y', text='text')

chart = ca_counties + nat_med + ca_med + la + CA_text + cou_text + nat_text + ca_text + la_text + y_label + line
chart = chart.configure_title(fontSize=22)

In [71]:
chart

alt.LayerChart(...)

In [72]:
chart.save('la_ca_nat_counties_hir.png', scale_factor=3)

---

## Fed Funds Rate

In [136]:
fed_plot = alt.Chart(fed, title='Target Fed Funds Rate'
                    ).mark_line(size=2.5
                               ).encode(x=alt.X('Date:T', 
                                                axis=alt.Axis(format='%Y', ticks=True, tickSize=10, values=[fed.Date[0], fed.Date[493], fed.Date[1013], fed.Date[1533]],
                                                              labelOpacity=.7, labelFontSize=18)), 
                                        y=alt.Y('Target_Rate', 
                                                title='Benchmark interest rate set by the Federal Reserve',
                                                axis=alt.Axis(values=list(np.arange(.01,.09,.01)), 
                                                              titleX=-30, titleY=-35, titleOpacity=.7, titleFontSize=20, 
                                                              labelOpacity=.7, labelFontSize=18, labelPadding=30, format='%')))

#text = alt.Chart(pd.DataFrame({'x':[0], 'y':[0], 'text':['1990']})).mark_text(dx=-290, dy=30, font='lato', fontSize=18, opacity=.7).encode(x=alt.X('x', axis=None), y='y', text='text')
#line = alt.Chart(pd.DataFrame({'x1':[fed.Date[0]], 'y1':[0], 'y2':[-.002]})).mark_rule(opacity=.7).encode(x=alt.X('x1:T', axis=alt.Axis(format='%Y')), y=alt.Y('y1', axis=None), y2='y2')
fed_chart = fed_plot

In [137]:
fed_chart

alt.Chart(...)

In [84]:
fed.Target_Rate = fed.Target_Rate * 100

In [ ]:
fed_plot = alt.Chart(fed, title='Target Fed Funds Rate'
                    ).mark_line(size=2.5
                               ).encode(x=alt.X('Date:T', 
                                                axis=alt.Axis(format='%Y', ticks=True, tickSize=10, 
                                                              values=[fed.Date[0], fed.Date[493], fed.Date[1013], fed.Date[1533]],
                                                              labelOpacity=.7, labelFontSize=18)), 
                                        y=alt.Y('Target_Rate', 
                                                title='Benchmark interest rate set by the Federal Reserve',
                                                axis=alt.Axis(values=list(range(1,8)), 
                                                              titleX=-25, titleY=-35, titleOpacity=.7, titleFontSize=20, 
                                                              labelOpacity=.7, labelFontSize=18, labelPadding=25),
                                                scale=alt.Scale(domain=[0,8])))

#text = alt.Chart(pd.DataFrame({'x':[0], 'y':[0], 'text':['1990']})).mark_text(dx=-290, dy=30, font='lato', fontSize=18, opacity=.7).encode(x=alt.X('x', axis=None), y='y', text='text')
#line = alt.Chart(pd.DataFrame({'x1':[fed.Date[0]], 'y1':[0], 'y2':[-.002]})).mark_rule(opacity=.7).encode(x=alt.X('x1:T', axis=alt.Axis(format='%Y')), y=alt.Y('y1', axis=None), y2='y2')
line = alt.Chart(pd.DataFrame({'x':[1990], 'y':[8], 'text':['_'*92]})).mark_text(dx=305, dy=-4, opacity=.09, fontSize=11.5, fontWeight='bold').encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')
y_labels_df = pd.DataFrame({'x':[1990], 'y':[8], 'text':['8%']})
y_label = alt.Chart(y_labels_df.query('x == 1990')).mark_text(dx=-13, dy=0, font='lato', fontSize=18, opacity=.7).encode(x=alt.X('x:O', axis=None), y=alt.Y('y', axis=None), text='text')

fed_chart = fed_plot + line + y_label

In [116]:
fed_plot

alt.Chart(...)

In [36]:
fed_chart.save('fed_funds_rate.png', scale_factor=3)

---

### CA Counties Housing Vacancy

In [6]:
ca = county5[['Year', 'FIPS', 'County', 'Population', 'Total_Units', 'Residents_Per_Unit', 'Vacant_Owned_Units_Percent', 'Vacant_Rental_Units_Percent', 'Vacant_Total_Units_Percent', 'Housing_Den', 'Pop_Den']][(county5.FIPS > 6000) & (county5.FIPS < 7000) & (county5.Population > 50000)]
ca_no_la = ca[~ca.FIPS.isin([6037])]
ca_la = ca[ca.FIPS == 6037]

In [15]:
ca_no_la_chart = alt.Chart(ca_no_la, title='CA Counties Vacant Housing'
                          ).mark_line(opacity=.7
                                     ).encode(x=alt.X('Year:O', axis=alt.Axis(ticks=True, tickSize=10, values=[2008, 2016])), 
                                              y=alt.Y('Vacant_Total_Units_Percent', 
                                                      title='Values represent owned and rental units', 
                                                      scale=alt.Scale(domain=[0,8]), 
                                                      axis=alt.Axis(values=list(range(1,9)))), 
                                              color=alt.Color('FIPS:O', scale=alt.Scale(range=['#e0e0e5']), legend=None))

ca_la_chart = alt.Chart(ca_la).mark_line(size=5, color='#fbcb42'
                                        ).encode(x='Year:O', 
                                                 y=alt.Y('Vacant_Total_Units_Percent', 
                                                         scale=alt.Scale(domain=[0,8]), 
                                                         axis=alt.Axis(values=list(range(1,9)))))

text_df = pd.DataFrame({'col1':[2008], 'col2':[8], 'col3':['%']})
text = alt.Chart(text_df).mark_text(dx=-36, dy=1, fontSize=16).encode(x='col1:O', y='col2', text='col3')

ca_vacancy = ca_no_la_chart + ca_la_chart + text

In [17]:
ca_vacancy

alt.LayerChart(...)

In [18]:
ca_vacancy.save('ca_counties_vacancy.png', scale_factor=3)

---

In [96]:
res08 = []
res16 = []
for county in ca.County.unique():
    fips_df = ca[ca.County == county]
    res08.append(fips_df.Residents_Per_Unit[fips_df.Year == 2008].iloc[0])
    res16.append(fips_df.Residents_Per_Unit[fips_df.Year == 2016].iloc[0])

In [97]:
changes_df = pd.DataFrame({'County':[c.split(',')[0].split(' County')[0] for c in ca.County.unique().tolist()], '08':res08, '16':res16})

In [98]:
changes_df.to_csv('ca_counties_res_per_unit0816.csv', index=False)  # Create DataWrapper arrow chart

In [20]:
changes = []
for county in ca.County.unique():
    fips_df = ca[ca.County == county]
    res08 = fips_df.Residents_Per_Unit[fips_df.Year == 2008].iloc[0]
    res16 = fips_df.Residents_Per_Unit[fips_df.Year == 2016].iloc[0]
    changes.append((res16 - res08) / res08 * 100)

In [21]:
changes_df = pd.DataFrame({'County':[c.split(',')[0].split(' County')[0] for c in ca.County.unique().tolist()], 'Change':changes})

In [22]:
changes_df.to_csv('ca_counties_res_per_unit_change_0816.csv', index=False)  # Create DataWrapper vertical bar plot

In [26]:
# Horizontal bar
alt.Chart(changes_df, title='CA Counties Housing Density Change').mark_bar().encode(x=alt.X('County:O', axis=alt.Axis(labelAngle=-90)), y=alt.Y('Change', title='Change in residents per unit from 2008 to 2016', axis=alt.Axis(values=[-5,0,5,10,15,20]), scale=alt.Scale(domain=[-10,20]))).properties(width=1000)

alt.Chart(...)